# Prédiction des tags

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)

datas = pd.read_csv("P5_06_transf_datas.csv")

datas

,stem_words,imp_words,Tags
0,convert decimal double c# want use track bar c...,convert decimal double c# want track bar chang...,c# floating-point type-conversion double ...
1,width collapse percentage width child element ...,width collapse percentage width child element ...,html css internet-explorer-7
2,calculate someone age base datetime type birth...,calculate someone age base datetime type give ...,c# .net datetime
3,calculate relative time c# give specific datet...,calculate relative time c# give specific datet...,c# datetime time datediff relative-time-s...
4,determine user timezone standards way web serv...,determine user timezone standards way web serv...,html browser timezone user-agent timezone...
...,...,...,...
29096,extract decimal part float point number extrac...,extract decimal part float point number extrac...,c floating-point decimal numbers
29097,flash load twice workaround investigate report...,flash load twice workaround investigate report...,flash swfobject
29098,etag vs header expire look around able figure ...,vs header expire look around able figure expir...,http caching etag expires-header
29099,determine longest similar portion several stri...,determine similar portion several string per t...,perl algorithm string similarity


In [2]:
samples = np.random.choice(len(datas), 5000, replace= False)
df_samples = datas.iloc[samples, :].reset_index(drop= True)

df_samples

,stem_words,imp_words,Tags
0,php push array may may exist want create array...,php push array may may exist want create array...,php arrays declaration push
1,correct way escape character datatable filter ...,correct way escape character datatable filter ...,.net ado.net
2,unpack array arguments php python provide oper...,unpack array arguments php python provide oper...,php python arguments iterable-unpacking
3,difference ole database odbc data source read ...,difference ole database odbc data source read ...,odbc oledb
4,get length jtextfield content user type jtextf...,get length content user type events seem time ...,java swing events jtextfield
...,...,...,...
4995,iis error cannot connect configuration databas...,iis error cannot connect configuration databas...,sql-server iis windows-server-2003
4996,python pep strongly recommend space tabs inden...,python strongly recommend space tabs indentati...,python indentation pep8
4997,return total number file within directory use ...,return total number file within directory php ...,php file count directory
4998,program interface write interface dns class ag...,program interface write interface dns class ag...,java model dns interface


Je sépare le jeu de données en un jeu d'entraînement et un de test.

In [3]:
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

X_docs = df_samples["imp_words"]
y_docs = df_samples["Tags"]

X_train, X_test, Y_train, Y_test = train_test_split(X_docs, y_docs, 
    train_size= 0.7, random_state= 42)



Je  transforme X et y.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

tfidf_vect = TfidfVectorizer(token_pattern= r'[\S]+')

x_train = tfidf_vect.fit_transform(X_train).toarray()
x_test = tfidf_vect.transform(X_test).toarray()


In [5]:
x_train.shape

(3500, 3959)

In [6]:
from sklearn.decomposition import PCA

component = 1000

pca = PCA(n_components= component)
pca.fit(x_train)

pca.explained_variance_ratio_.cumsum()[-1]

0.8311071312563882

In [7]:
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

In [8]:
count_vect = CountVectorizer(token_pattern= r'[\S]+')
y_train = count_vect.fit_transform(Y_train).toarray()

y_test = count_vect.transform(Y_test).toarray()


In [9]:
# from sklearn.preprocessing import MinMaxScaler

# mms = MinMaxScaler()
# mms.fit(x_train_pca)

# x_train_mms = mms.transform(x_train_pca)
# x_test_mms = mms.transform(x_test_pca)

In [11]:
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier

# clf = OneVsRestClassifier(LogisticRegression( class_weight= "balanced", 
#     random_state= 42)).fit(x_train_pca, y_train)

# clf = OneVsRestClassifier(SGDClassifier(loss= "modified_huber", # class_weight= "balanced", 
#     random_state= 42))
# clf.fit(x_train_pca, y_train)

clf = OneVsRestClassifier(RandomForestClassifier()).fit(x_train_pca, y_train)

# clf = OneVsRestClassifier(MultinomialNB()).fit(x_train_mms, y_train)

In [13]:
clf.score(x_test_pca, y_test)

0.013333333333333334

In [ ]:
# dec_funct = clf.decision_function(x_test_pca)

In [ ]:
# np.argmax(dec_funct[1])

In [ ]:
# max(dec_funct[1])

In [15]:
pred = clf.predict(x_test_pca)
pred_proba = clf.predict_proba(x_test_pca)

In [ ]:
# pred.shape

In [ ]:
# # pred.shape
len(np.where(np.max(pred, axis= 1) == 1)[0])

TypeError: cannot perform reduce with flexible type

In [ ]:
truc = np.sort(pred_proba, axis= 1)
print(truc[-1][-10:])

diff_tt = truc[0, -1] - truc[:, -2]
len(diff_tt)

In [ ]:
for t in reversed(truc[0]):
    print(t, " - ", np.log10(t+1))

In [ ]:
tt = diff_tt.reshape(-1, 1)
tt.shape

In [ ]:
np.where(pred_proba > diff_tt.reshape(-1, 1), 1, 0)

# maxi = max(pred[0])

# print(maxi)
# np.argmax(pred[0])

In [ ]:
# machin = sorted(pred[0], reverse= True)

# print(machin[0] - machin[1])
# print(maxi)
# puissance = np.log10(maxi).astype(int).astype(float) -1
# print(puissance)
# print(1 * 10**(puissance))
# maxi - (2 * 10**(puissance))

In [ ]:
# truc = np.where(pred[0] >= 0.5, 1, 0)
# truc = truc.reshape(1, -1)
# truc.shape

In [16]:
results_preds = np.zeros(pred.shape)

# for num, ind in enumerate(pred):
pred_sorted = np.sort(pred_proba, axis= 1)
diff = pred_sorted[:, -1] - np.log10(pred_sorted[:, -1] +1)
    # transformed_vals = np.where(ind > (maxi - limit), 1, 0).reshape(1, -1)
# results_preds = np.where(maxi - limit > 0, np.where(pred_proba > (maxi - limit), 1, 0), 0) #np.where(pred_proba == maxi, 1, 0))
results_preds = np.where(pred_proba > diff.reshape(-1, 1), 1, 0)
    # print(transformed_vals.shape)
    # results_preds[num] = transformed_vals
    # if num >2:
    #     break

# results_preds = np.where(pred_proba > 0, 1, 0)

In [17]:
results_preds.shape
# np.max(results_preds, axis= 1)

(1500, 2372)

In [18]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import roc_auc_score, precision_recall_curve

np.mean(y_test == results_preds)


0.9984367622259697

In [20]:
np.mean(y_test == pred)

0.9989078133783024

0.99191208791
0.99869461

In [21]:
t_pred = pd.DataFrame(pred_proba, columns= count_vect.get_feature_names_out())


In [22]:
pred_proba_df = pd.DataFrame(results_preds, columns= count_vect.get_feature_names_out())

In [23]:
from collections import defaultdict

filtered_pred = defaultdict(int)

ind= 0

for col in pred_proba_df.columns:
    if pred_proba_df.loc[ind, col] > diff[ind]:
        filtered_pred[col] = pred_proba_df.loc[ind, col]


In [ ]:
filtered_pred = defaultdict(int)

ind= 451

for col in t_pred.columns:
    if t_pred.loc[ind, col] > diff_tt:
        filtered_pred[col] = t_pred.loc[ind, col]

In [24]:
x_test_df = pd.DataFrame(x_test, columns= tfidf_vect.get_feature_names_out())
y_test_df = pd.DataFrame(y_test, columns= count_vect.get_feature_names_out())

In [25]:
filtered_pred

defaultdict(int, {'python': 1})

In [26]:
test = y_test_df.iloc[ind, :].reset_index().rename({ind: 0}, axis= 1)
test[test[0] > 0]
# test

,index,0
496,datetime,1
1588,python,1
1710,rfc3339,1


In [27]:
X_test.iloc[ind]

'parse iso date need parse rfc string like 2008-09 python datetime type find python standards library convenient best way'